In [22]:
import pandas as pd
import tensorflow_hub as hub
import numpy as np
import json 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

In [23]:
csv_file_path = "Data/Kaggle/chatroom.csv"
chunksize = 10**2

In [24]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
embed = hub.KerasLayer(module_url)

In [25]:
msdialog_filepath = "data/MSDialog/Intent/MSDialog-Intent.json"
with open(msdialog_filepath) as f:
    data = json.load(f)

print(len(data))
message_list = []
labels = []
for j, thread in enumerate(data.keys()):
    messages = data[thread]["utterances"]
    for i, message in enumerate(messages):
        message_list.append(message["utterance"])
        labels.append(message["tags"].split(" "))
    if j == 300:
        break

2199


In [26]:
print("Number of messages: {}".format(len(message_list)))

Number of messages: 1390


In [64]:
one_hot = MultiLabelBinarizer()
multi_hot_labels = one_hot.fit_transform(labels)

In [28]:
message_embeddings = np.array(embed.call(message_list))
N, D = np.shape(message_embeddings)
print("Dimensionality: {}\nNumber of messages: {}".format(D,N))
_, L = multi_hot_labels.shape
print("{} Unique Classes".format(L))

Dimensionality: 512
Number of messages: 1390
13 Unique Classes


In [29]:
# Neural network
model = Sequential()
model.add(Dense(256, input_dim=D, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#model.add(Dense(128, activation="relu"))
#model.add(Dense(64, activation="relu"))
#model.add(Dense(32, activation="relu"))

model.add(Dense(16, activation="relu"))
model.add(BatchNormalization())
model.add(Dense(L, activation="softmax"))

In [30]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(message_embeddings, 
                                                    multi_hot_labels, test_size=0.30, random_state=42)

In [35]:
history = model.fit(X_train, y_train, epochs=100, batch_size=100, validation_data=(X_test,y_test))

Train on 973 samples, validate on 417 samples
Epoch 1/100
973/973 [==============================] - 0s 59us/sample - loss: 0.0515 - accuracy: 0.6608 - val_loss: 0.0815 - val_accuracy: 0.4484
Epoch 2/100
973/973 [==============================] - 0s 60us/sample - loss: 0.0513 - accuracy: 0.6351 - val_loss: 0.0808 - val_accuracy: 0.4388
Epoch 3/100
973/973 [==============================] - 0s 53us/sample - loss: 0.0515 - accuracy: 0.6280 - val_loss: 0.0802 - val_accuracy: 0.4604
Epoch 4/100
973/973 [==============================] - 0s 59us/sample - loss: 0.0512 - accuracy: 0.6290 - val_loss: 0.0801 - val_accuracy: 0.4532
Epoch 5/100
973/973 [==============================] - 0s 50us/sample - loss: 0.0511 - accuracy: 0.6341 - val_loss: 0.0790 - val_accuracy: 0.4365
Epoch 6/100
973/973 [==============================] - 0s 49us/sample - loss: 0.0510 - accuracy: 0.6351 - val_loss: 0.0777 - val_accuracy: 0.4556
Epoch 7/100
973/973 [==============================] - 0s 50us/sample - loss: 

In [36]:
example = X_test[0:20][:]
y_pred = model.predict(example)
y_true = y_test[0:20]

In [38]:
i = 10
trueVals = np.where(y_true[i] == 1)[0]
topN = np.size(np.where(y_true[i] == 1))

predictions = (-y_pred[i]).argsort()[0:topN]
topProb = np.abs(np.sort(-y_pred[i])[0:topN])
Uncertainty = (1 - np.sum(topProb)) * 100

print("True: {}".format(trueVals))
print("Pred: {}".format(predictions))
print("Probabilities: {}".format(topProb))
print("Uncertainty: {} % ".format(Uncertainty))

True: [ 1  2 12]
Pred: [9 2 7]
Probabilities: [0.6831202  0.17580204 0.03836461]
Uncertainty: 10.271316766738892 % 


In [57]:
test = ["Hello FCC", "FUCK OFF!"]
test_embeddings = np.array(embed.call(test))
out = model.predict(test_embeddings)
print(out)
print(np.argmax(out[0]))
print(one_hot.classes_)

[[4.17918200e-03 3.46101820e-02 7.36351252e-01 3.32589913e-03
  2.52956408e-03 5.16425632e-03 1.21750869e-03 1.25192478e-02
  2.28362251e-03 8.74203106e-04 1.17678419e-01 1.12987375e-02
  6.79679587e-02]
 [1.89263932e-03 2.12365668e-02 7.86002143e-04 9.88672909e-05
  5.29983127e-03 1.18867140e-02 3.26954946e-03 4.13225032e-04
  2.82672467e-03 2.11244187e-04 9.48920608e-01 5.03680552e-04
  2.65441253e-03]]
2
['' 'CQ' 'FD' 'FQ' 'GG' 'IR' 'JK' 'NF' 'O' 'OQ' 'PA' 'PF' 'RQ']


In [62]:
multi_hot_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [65]:
one_hot.classes_

array(['', 'CQ', 'FD', 'FQ', 'GG', 'IR', 'JK', 'NF', 'O', 'OQ', 'PA',
       'PF', 'RQ'], dtype=object)